# MPI Send and Receive

Sending and receiving data with MPI.

See [the original tutorial](https://mpitutorial.com/tutorials/mpi-send-and-receive/) (MIT License) for more narrative detail and examples in C.


## Initializing the cluster

As with all tutorials, we will start by creating a cluster and loading the rank and size.

In [12]:
import logging
import ipyparallel as ipp

# create a cluster
rc = ipp.Cluster(engines="mpi", n=2, log_level=logging.WARNING).start_and_connect_sync(activate=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.50engine/s]


In [4]:
%%px
# Find out rank, size
from mpi4py import MPI

comm = MPI.COMM_WORLD
world_rank = comm.Get_rank()
world_size = comm.Get_size()

print(f"I am rank {world_rank} / {world_size}")

[stdout:1] I am rank 1 / 2


[stdout:0] I am rank 0 / 2


## MPI send/recv 

[original tutorial](https://mpitutorial.com/tutorials/mpi-send-and-receive/#mpi-send--recv-program)

In [5]:
%%px

number = None

if world_rank == 0:
    number = -1
    comm.send(number, dest=1)
elif world_rank == 1:
    number = comm.recv(source=0)
    print(f"Process 1 received number {number} from process 0")

number

Out[0:2]: -1

[stdout:1] Process 1 received number -1 from process 0


Out[1:2]: -1

## MPI ping pong program

[original tutorial](https://mpitutorial.com/tutorials/mpi-send-and-receive/#mpi-ping-pong-program)


In [8]:
%%px

import time

PING_PONG_LIMIT = 5

ping_pong_count = 0
partner_rank = (world_rank + 1) % 2

while ping_pong_count < PING_PONG_LIMIT:
    if world_rank == ping_pong_count % 2:
        ping_pong_count += 1
        comm.send(ping_pong_count, dest=partner_rank)
        print(f"{world_rank} sent and incremented ping_pong_count {ping_pong_count} to {partner_rank}")
    else:
        ping_pong_count = comm.recv(source=partner_rank)
        print(f"{world_rank} received ping_pong_count {ping_pong_count} from {partner_rank}")

    # Make sure the output is synchronized
    # (this is not necessary in IPython Parallel)
    comm.Barrier()


[stdout:1] 1 received ping_pong_count 1 from 0
1 sent and incremented ping_pong_count 2 to 0
1 received ping_pong_count 3 from 0
1 sent and incremented ping_pong_count 4 to 0
1 received ping_pong_count 5 from 0


[stdout:0] 0 sent and incremented ping_pong_count 1 to 1
0 received ping_pong_count 2 from 1
0 sent and incremented ping_pong_count 3 to 1
0 received ping_pong_count 4 from 1
0 sent and incremented ping_pong_count 5 to 1


## Ring program


[original tutorial](https://mpitutorial.com/tutorials/mpi-send-and-receive/#ring-program)


This one uses more than 2 processes, so stop our first cluster and start a new one.

In [10]:
try:
    rc.cluster.stop_cluster_sync()
except NameError:
    # rc undefined, e.g. not starting from scratch
    pass

# start a cluster and connect to it
rc = ipp.Cluster(engines="mpi", n=4).start_and_connect_sync(activate=True)

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 38004, 'identifier': 'ipcontroller-1681285126-r1fp-37933'}
Stopping engine(s): 1681285128
engine set stopped 1681285128: {'exit_code': 0, 'pid': 38018, 'identifier': 'ipengine-1681285126-r1fp-1681285128-37933'}
Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.78engine/s]


In [11]:
%%px

from mpi4py import MPI

comm = MPI.COMM_WORLD
world_rank = comm.Get_rank()
world_size = comm.Get_size()

if world_rank != 0:
    token = comm.recv(source=world_rank - 1)
    print(f"{world_rank} received {token} from {world_rank-1}")
    token = token + f"{world_rank}".encode('ascii')
else:
    token = b"0"

comm.send(token, dest=(world_rank + 1) % world_size)

if world_rank == 0:
    token = comm.recv(source=world_size - 1)
    print(f"{world_rank} received {token} from {world_size-1}")

[stdout:1] 1 received b'0' from 0


[stdout:2] 2 received b'01' from 1


[stdout:0] 0 received b'0123' from 3


[stdout:3] 3 received b'012' from 2


In [13]:
rc.cluster.stop_cluster_sync()